# L5: Automate Event Planning

In this lesson, you will learn more about Tasks.

The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [2]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import libraries, APIs and LLM

In [3]:
from crewai import Agent, Crew, Task

**Note**: 
- The video uses `gpt-4-turbo`, but due to certain constraints, and in order to offer this course for free to everyone, the code you'll run here will use `gpt-3.5-turbo`.
- You can use `gpt-4-turbo` when you run the notebook _locally_ (using `gpt-4-turbo` will not work on the platform)
- Thank you for your understanding!

In [4]:
import os
from utils import get_openai_api_key,get_serper_api_key

openai_api_key = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'
os.environ["SERPER_API_KEY"] = get_serper_api_key()

## crewAI Tools

In [5]:
from crewai_tools import ScrapeWebsiteTool, SerperDevTool

# Initialize the tools
search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()

## Creating Agents

In [6]:
# Agent 1: Venue Coordinator
venue_coordinator = Agent(
    role="Venue Coordinator",
    goal="Identify and book an appropriate venue "
    "based on event requirements",
    tools=[search_tool, scrape_tool],
    verbose=True,
    backstory=(
        "With a keen sense of space and "
        "understanding of event logistics, "
        "you excel at finding and securing "
        "the perfect venue that fits the event's theme, "
        "size, and budget constraints."
    )
)

In [7]:
 # Agent 2: Logistics Manager
logistics_manager = Agent(
    role='Logistics Manager',
    goal=(
        "Manage all logistics for the event "
        "including catering and equipmen"
    ),
    tools=[search_tool, scrape_tool],
    verbose=True,
    backstory=(
        "Organized and detail-oriented, "
        "you ensure that every logistical aspect of the event "
        "from catering to equipment setup "
        "is flawlessly executed to create a seamless experience."
    )
)

In [8]:
# Agent 3: Marketing and Communications Agent
marketing_communications_agent = Agent(
    role="Marketing and Communications Agent",
    goal="Effectively market the event and "
         "communicate with participants",
    tools=[search_tool, scrape_tool],
    verbose=True,
    backstory=(
        "Creative and communicative, "
        "you craft compelling messages and "
        "engage with potential attendees "
        "to maximize event exposure and participation."
    )
)

## Creating Venue Pydantic Object

- Create a class `VenueDetails` using [Pydantic BaseModel](https://docs.pydantic.dev/latest/api/base_model/).
- Agents will populate this object with information about different venues by creating different instances of it.

In [9]:
from pydantic import BaseModel
# Define a Pydantic model for venue details 
# (demonstrating Output as Pydantic)
class VenueDetails(BaseModel):
    name: str
    address: str
    capacity: int
    booking_status: str

## Creating Tasks
- By using `output_json`, you can specify the structure of the output you want.
- By using `output_file`, you can get your output in a file.
- By setting `human_input=True`, the task will ask for human feedback (whether you like the results or not) before finalising it.

In [10]:
venue_task = Task(
    description="Find a venue in {event_city} "
                "that meets criteria for {event_topic}.",
    expected_output="All the details of a specifically chosen"
                    "venue you found to accommodate the event.",
    human_input=True,
    output_json=VenueDetails,
    output_file="venue_details.json",  
      # Outputs the venue details as a JSON file
    agent=venue_coordinator
)

- By setting `async_execution=True`, it means the task can run in parallel with the tasks which come after it.

In [11]:
logistics_task = Task(
    description="Coordinate catering and "
                 "equipment for an event "
                 "with {expected_participants} participants "
                 "on {tentative_date}.",
    expected_output="Confirmation of all logistics arrangements "
                    "including catering and equipment setup.",
    human_input=True,
    async_execution=False, # all tasks AFTERWARDs are in parallel
    agent=logistics_manager
)

In [12]:
marketing_task = Task(
    description="Promote the {event_topic} "
                "aiming to engage at least"
                "{expected_participants} potential attendees.",
    expected_output="Report on marketing activities "
                    "and attendee engagement formatted as markdown.",
    async_execution=False,
    output_file="marketing_report.md",  # Outputs the report as a text file
    agent=marketing_communications_agent
)

## Creating the Crew

**Note**: Since you set `async_execution=True` for `logistics_task` and `marketing_task` tasks, now the order for them does not matter in the `tasks` list.

In [13]:
# Define the crew with agents and tasks
event_management_crew = Crew(
    agents=[venue_coordinator, 
            logistics_manager, 
            marketing_communications_agent],
    
    tasks=[venue_task, 
           logistics_task, 
           marketing_task],
    
    verbose=True
)

## Running the Crew

- Set the inputs for the execution of the crew.

In [14]:
event_details = {
    'event_topic': "Tech Innovation Conference",
    'event_description': "A gathering of tech innovators "
                         "and industry leaders "
                         "to explore future technologies.",
    'event_city': "San Francisco",
    'tentative_date': "2024-09-15",
    'expected_participants': 500,
    'budget': 20000,
    'venue_type': "Conference Hall"
}

**Note 1**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

**Note 2**: 
- Since you set `human_input=True` for some tasks, the execution will ask for your input before it finishes running.
- When it asks for feedback, use your mouse pointer to first click in the text box before typing anything.

In [15]:
result = event_management_crew.kickoff(inputs=event_details)

 [DEBUG]: == Working Agent: Venue Coordinator
 [INFO]: == Starting Task: Find a venue in San Francisco that meets criteria for Tech Innovation Conference.


> Entering new CrewAgentExecutor chain...
I need to find a venue in San Francisco that is suitable for a Tech Innovation Conference. I must consider the theme, size, and budget constraints of the event.

Action: Search the internet
Action Input: {"search_query": "tech event venues in San Francisco"} 


Search results: Title: The 16 Best Networking Venues for Rent in San Francisco, CA
Link: https://www.tagvenue.com/us/hire/networking-venues/san-francisco
Snippet: Known for its tech and startup scene, SoMa provides modern and creative spaces ideal for tech networking events. Attractions like the San Francisco Museum ...
---
Title: 10 of the Top Event and Co-Working Spaces in the Bay Area
Link: https://www.startupgrind.com/blog/10-of-the-top-event-and-co-working-spaces-in-the-bay-area/
Snippet: 1. HanaHaus, Palo Alto: · 2. Pivotal Lab

This is the agent final answer: After conducting research, I have found a variety of networking venues in San Francisco that could be suitable for the Tech Innovation Conference. Some options include:
1. El Lopo in Van Ness, San Francisco, CA - A convivial neighborhood bar with a relaxed and approachable atmosphere, suitable for events ranging from karaoke to trivia.
2. The Newhall Nest in Financial District, San Francisco, CA - A rooftop space with premium amenities like a commercial bar set up and Trager grill, perfect for accommodating 30+ guests for various events.
3. Alloy Collective in South of Market (SOMA), San Francisco, CA - A wide-open industrial space with high ceilings, two bathrooms, a kitchenette, and ground-floor access, ideal for a wide range of uses.
4. Chambers at the Phoenix in Tenderloin, San Francisco, CA - A lounge space that can accommodate 20-150 guests and features a full bar, DJ booth, and an 80 flat-screen television.

These venues offer different capacities

Thought: 
Based on the search results, I can see that the key selling points for the Tech Innovation Conference are:
- Gain new industry insights
- Build a valuable network
- Find solutions to real problems
- Learn from experts and peers
- Networking opportunities
- Improve skills
- Discover new software tool capabilities
- Learn about shortcomings in software tools

Action: Delegate work to co-worker
Action Input: {"coworker": "Venue Coordinator", "task": "Confirm availability of The Pearl venue for 500 participants on 2024-09-15", "context": "I need to confirm if The Pearl venue in the Dogpatch neighborhood of San Francisco can accommodate 500 participants for the Tech Innovation Conference on 2024-09-15."}

> Entering new CrewAgentExecutor chain...
I need to gather information about The Pearl venue in the Dogpatch neighborhood of San Francisco to confirm its availability for the Tech Innovation Conference on 2024-09-15.

Action: Read website content
Action Input: {"website_url": "ht

Thought: 
Now that I have identified key marketing strategies and attendee engagement ideas, I need to compile a report on marketing activities and attendee engagement for the Tech Innovation Conference.

Action: Read website content
Action Input: {"website_url": "https://globalconference.ca/what-are-benefits-of-innovation-conference/"} 

What Are Benefits of Innovation Conference? 
Skip to contentNotre service client est désormais disponible en français. Vous pouvez nous contacter au numéro de service client +1 672-971-4036 ou par email à l'adresse info@globalconference.ca
Home
Conferences
Menu Toggle
Upcoming Conferences
Menu Toggle
Upcoming Global Conference in Canada
Global Conference in USA 2025
Conference in French Language
Past Conferences
Keynote Speakers
Conference Committee
Conferences Held On
Webinar
Journal
Menu Toggle
GBEJ
Publication
Menu Toggle
Conference Abstracts
Conference Proceedings
Author Paper Submission
Peer Review Details
Presentation
Menu Toggle
Modes Of Presen

Final Answer: 
# Marketing Activities Report:

## Key Selling Points for the Tech Innovation Conference:
- Gain new industry insights
- Build a valuable network
- Find solutions to real problems
- Learn from experts and peers
- Networking opportunities
- Improve skills
- Discover new software tool capabilities
- Learn about shortcomings in software tools

## Marketing Strategies for Attendee Engagement:
- Implement new strategies from conference learnings
- Use innovative event marketing strategies to attract and engage attendees
- Utilize disruptive technologies like VR/AR and AI in event marketing
- Explore virtual event tips for boosting engagement
- Focus on clear and compelling branding for successful virtual conferences
- Incorporate interactive booths and hands-on activities for attendee engagement
- Create a multi-faceted marketing strategy for tech-based events
- Leverage clear and consistent messaging to promote the event

## Attendee Engagement Benefits:
- Gain new industry 

- Display the generated `venue_details.json` file.

In [18]:
import json
from pprint import pprint

with open('venue_details.json') as f:
   data = json.load(f)

pprint(data)

{'address': 'Van Ness, San Francisco, CA',
 'booking_status': 'Available',
 'capacity': 100,
 'name': 'El Lopo'}


- Display the generated `marketing_report.md` file.

**Note**: After `kickoff` execution has successfully ran, wait an extra 45 seconds for the `marketing_report.md` file to be generated. If you try to run the code below before the file has been generated, your output would look like:

```
marketing_report.md
```

If you see this output, wait some more and than try again.

In [20]:
from IPython.display import Markdown
Markdown("marketing_report.md")

# Marketing Activities Report:

## Key Selling Points for the Tech Innovation Conference:
- Gain new industry insights
- Build a valuable network
- Find solutions to real problems
- Learn from experts and peers
- Networking opportunities
- Improve skills
- Discover new software tool capabilities
- Learn about shortcomings in software tools

## Marketing Strategies for Attendee Engagement:
- Implement new strategies from conference learnings
- Use innovative event marketing strategies to attract and engage attendees
- Utilize disruptive technologies like VR/AR and AI in event marketing
- Explore virtual event tips for boosting engagement
- Focus on clear and compelling branding for successful virtual conferences
- Incorporate interactive booths and hands-on activities for attendee engagement
- Create a multi-faceted marketing strategy for tech-based events
- Leverage clear and consistent messaging to promote the event

## Attendee Engagement Benefits:
- Gain new industry insights
- Build a valuable network
- Find solutions to real problems
- Explore new technologies
- Recharge and get inspired
- Expand business opportunities

## Innovation Conference Benefits:
- Fresh insights
- Networking possibilities
- Knowledge exchange
- Inspiration and motivation
- Access to resources

## Innovation Conference FAQs:
- Attending can help solve specific challenges in work
- Enhances professional development
- Benefits organizations by driving growth and success
- Provides intangible benefits like inspiration and motivation
- Expands knowledge beyond current expertise
- Contributes to professional reputation and credibility

---

Overall, the marketing activities for the Tech Innovation Conference focus on highlighting key selling points, implementing effective marketing strategies for attendee engagement, and showcasing the benefits of attending innovation conferences. The goal is to attract and engage potential attendees by emphasizing the value and opportunities that the conference offers.